In [1]:
# %load_ext autoreload
# %autoreload 2


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

# from utils import evaluate_binary, log_mlflow

In [4]:
from typing import Optional
from pathlib import Path

def find_project_root() -> Optional[Path]:
    current = Path(".").resolve()
    
    while True:
        if (current / ".git").exists():
            return current
        
        if current.parent == current:
            print("WARNING: No .git dir found")
            return current
              
        current = current.parent
        

PROJECT_ROOT = find_project_root()
PROJECT_ROOT

WindowsPath('C:/Users/Harsha/Documents/ISB_AMPBA/Term5/FP2/GroupAssignment/project')

In [5]:
# Tag this cell as 'parameters'
# BASE
COUNTRY = 'US'

dataset_dir  = "datadir\\datasets"
model_dir  = "model\\linear"
transformed_data_dir = "datadir\\transformed"

In [6]:
# Parameters
COUNTRY = "RU"


### Import data

In [7]:
df = pd.read_parquet(f'{PROJECT_ROOT}\\{dataset_dir}\\{COUNTRY}.parquet')
# df["stock_name"] = INDEX
df.head()

,Date,production
0,2002-01-01,7001
1,2002-02-01,7094
2,2002-03-01,7143
3,2002-04-01,7135
4,2002-05-01,7215


### Data prep

In [8]:
df.sort_values(by='Date', ascending=True, inplace=True)  # Have to sort ascending for rolling to work correctly

In [9]:
df['Date'] =  pd.to_datetime(df['Date'])

### Create basic features

In [10]:
df['prev'] = df['production'].shift(1)
df.head()

,Date,production,prev
0,2002-01-01,7001,NaN
1,2002-02-01,7094,7001.0
2,2002-03-01,7143,7094.0
3,2002-04-01,7135,7143.0
4,2002-05-01,7215,7135.0


In [11]:
df['sma2'] = df['prev'].rolling(window=2).mean()
df['sma5'] = df['prev'].rolling(window=5).mean()
df['sma10'] = df['prev'].rolling(window=10).mean()
df['sma20'] = df['prev'].rolling(window=20).mean()
df.head()

,Date,production,prev,sma2,sma5,sma10,sma20
0,2002-01-01,7001,NaN,NaN,NaN,NaN,NaN
1,2002-02-01,7094,7001.0,NaN,NaN,NaN,NaN
2,2002-03-01,7143,7094.0,7047.5,NaN,NaN,NaN
3,2002-04-01,7135,7143.0,7118.5,NaN,NaN,NaN
4,2002-05-01,7215,7135.0,7139.0,NaN,NaN,NaN


In [12]:
assert round(df['prev'].tail(5).mean(), 4) == round(df['sma5'].tail(1).item(), 4), 'Expected sma5 to be same as mean of past 5 items'
assert round(df['prev'].tail(10).mean(), 4) == round(df['sma10'].tail(1).item(), 4), 'Expected sma10 to be same as mean of past 10 items'
assert round(df['prev'].tail(20).mean(), 4) == round(df['sma20'].tail(1).item(), 4), 'Expected sma20 to be same as mean of past 20 items'

In [13]:
df['ema12'] = df['prev'].ewm(span=12, min_periods=12, adjust=False).mean()
df['ema26'] = df['prev'].ewm(span=26, min_periods=26, adjust=False).mean()

In [14]:
def difference(prev_price, moving_average):
    return (prev_price - moving_average) / prev_price

In [15]:
for col in ['sma2', 'sma5', 'sma10', 'sma20', 'ema12', 'ema26']:
    df['{}_diff'.format(col)] = difference(df['prev'], df[col])

In [16]:
df.dropna(inplace=True)

In [17]:
df.shape

(225, 15)

In [18]:
# df.drop(columns=['date', 'open'], inplace=True)

### Create label

In [19]:
df['label'] = (df['production'] > df['prev']).astype(int)
df.reset_index(inplace = True)
# df.drop(['index'], axis=1)


In [20]:
df = df.drop(['index'], axis=1)
df

,Date,production,prev,sma2,sma5,sma10,sma20,ema12,ema26,sma2_diff,sma5_diff,sma10_diff,sma20_diff,ema12_diff,ema26_diff,label
0,2004-03-01,8661,8599.0,8570.5,8537.8,8413.9,8062.90,8323.232091,7974.601336,0.003314,0.007117,0.021526,0.062344,0.032070,0.072613,1
1,2004-04-01,8729,8661.0,8630.0,8566.2,8473.2,8123.40,8375.196384,8025.445681,0.003579,0.010946,0.021683,0.062071,0.032999,0.073381,1
2,2004-05-01,8805,8729.0,8695.0,8610.0,8527.1,8182.55,8429.627710,8077.560816,0.003895,0.013633,0.023130,0.062602,0.034296,0.074629,1
3,2004-06-01,8974,8805.0,8767.0,8667.2,8574.6,8241.50,8487.377293,8131.445200,0.004316,0.015650,0.026167,0.063998,0.036073,0.076497,1
4,2004-07-01,9017,8974.0,8889.5,8753.6,8634.3,8305.75,8562.242325,8193.856667,0.009416,0.024560,0.037854,0.074465,0.045883,0.086934,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,2022-07-01,10019,9984.0,9715.0,9833.6,10001.7,9803.05,9873.905983,9897.523350,0.026943,0.015064,-0.001773,0.018124,0.011027,0.008662,1
221,2022-08-01,9995,10019.0,10001.5,9791.2,9998.4,9838.70,9896.228139,9906.521620,0.001747,0.022737,0.002056,0.017996,0.012254,0.011227,0
222,2022-09-01,9988,9995.0,10007.0,9753.8,9982.8,9872.35,9911.423810,9913.075574,-0.001201,0.024132,0.001221,0.012271,0.008362,0.008197,0
223,2022-10-01,9881,9988.0,9991.5,9886.4,9963.2,9898.70,9923.204763,9918.625532,-0.000350,0.010172,0.002483,0.008941,0.006487,0.006946,0


### Prepare dataset to store in parquet format

In [21]:
df.dtypes

Date          datetime64[ns]
production             int64
prev                 float64
sma2                 float64
sma5                 float64
sma10                float64
sma20                float64
ema12                float64
ema26                float64
sma2_diff            float64
sma5_diff            float64
sma10_diff           float64
sma20_diff           float64
ema12_diff           float64
ema26_diff           float64
label                  int32
dtype: object

In [22]:

predictors_df = df.loc[:,df.columns!="label"]
target_df = df[["label","Date"]]

datalen = len(df)
idslist= list(range(datalen))

record_ids = pd.DataFrame(data = idslist, columns = ["record_id"])

predictors_df = pd.concat(objs = [predictors_df, record_ids], axis = 1)
target_df = pd.concat(objs = [target_df, record_ids], axis = 1)

predictors_df.rename(columns = {'Date':'event_timestamp','production': 'oil_production'}, inplace = True)
target_df.rename(columns = {'Date':'event_timestamp','production': 'oil_production'}, inplace = True)

In [23]:
target_df.head()
target_df.dtypes

label                       int32
event_timestamp    datetime64[ns]
record_id                   int64
dtype: object

In [24]:
# Save FE

Path(f"{PROJECT_ROOT}\\{transformed_data_dir}\\{COUNTRY}").mkdir(parents=True, exist_ok=True)
predictors_df.to_parquet(f"{PROJECT_ROOT}\\{transformed_data_dir}\\{COUNTRY}\\{COUNTRY}_features.parquet",index=False)
target_df.to_parquet(f"{PROJECT_ROOT}\\{transformed_data_dir}\\{COUNTRY}\\{COUNTRY}_target.parquet",index=False)
